In [ ]:
# Se importan todas las librerias que usaremos
import requests
import json
from IPython.display import JSON
from keys import *

In [ ]:
# Este es el endpoint que usaremos para obtener los vuelos
url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/LEBL/2025-09-01T08:00/2025-09-01T20:00"

# Estos son los parametros que usaremos en la consulta
querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}

# Estos son los headers que usaremos en la consulta
headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
    'x-rapidapi-key': flights_key
    }

# Se llama a la API o consulta
response = requests.request("GET", url, headers=headers, params=querystring)


In [ ]:
# Se verifica que la respuesta sea correcta y se guarda en un archivo JSON
if response.status_code == 200:
    new_data = response.json()

    try:
        with open("data.json", "r") as json_file:
            existing_data = json.load(json_file)
    except (FileNotFoundError, json.decoder.JSONDecodeError):
        existing_data = []

    existing_data.append(new_data)

    with open("data.json", "w") as json_file:
        json.dump(existing_data, json_file, indent=4)
        print("Data appended to data.json file.")
else:
    print("Failed to retrieve data from the API. Status code:", response.status_code)

Data appended to data.json file.


In [ ]:
# Se carga el archivo JSON
flight_arrivals = response.json()

# Se imprimen los datos de interes
for flight in flight_arrivals:
    flight_arrivals['arrivals'][0]['arrival']['scheduledTime']
    flight_arrivals['arrivals'][0]['number']
    flight_arrivals['arrivals'][0]['arrival']['terminal']
    flight_arrivals['arrivals'][0]['departure']['airport']['name']
    flight_arrivals['arrivals'][0]['aircraft']['model']
    flight_arrivals['arrivals'][0]['airline']['name']

In [ ]:
import pandas as pd
from numpy import nan

# Se carga el archivo JSON
flight_arrivals = response.json()

# Se crea una funcion para extraer los datos de interes y convertirlos en un dataframe
def flight_extraction(flights):
    flights_df = pd.json_normalize({
        'Scheduled_arrival_time': flights['arrival']['scheduledTime'],
        'flight_number': flights['number'],
        'from' : flights['departure']['airport']['name'],
        'airline' : flights['airline']['name'],
        'aircraft' : flights['aircraft']['model']
    })
    return(flights_df)

# Se aplica la funcion a todos los vuelos y se concatenan en un solo dataframe
flight_arrivals = pd.concat(
    [flight_extraction(flight) for flight in flight_arrivals['arrivals']]
)

# Se muestran los primeros 5 registros del dataframe
flight_arrivals.head()


,flight_number,from,airline,aircraft,Scheduled_arrival_time.utc,Scheduled_arrival_time.local
0,VY 3907,Palma De Mallorca,Vueling,Airbus A320,2025-09-01 06:20Z,2025-09-01 08:20+02:00
0,IB 5127,Palma De Mallorca,Iberia,Airbus A320,2025-09-01 06:20Z,2025-09-01 08:20+02:00
0,LA 5876,Palma De Mallorca,LATAM,Airbus A320,2025-09-01 06:20Z,2025-09-01 08:20+02:00
0,V7 2344,Brest/Guipavas,Volotea,Airbus A320,2025-09-01 06:15Z,2025-09-01 08:15+02:00
0,IB 5153,Rome,Iberia,Airbus A320 NEO,2025-09-01 06:15Z,2025-09-01 08:15+02:00
